In [8]:
import logging
from pathlib import Path
import yaml
from ultralytics import YOLO

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def train_yolo(
    data_yaml: str,
    model: str = "yolov8n.pt",
    epochs: int = 40,
    batch: int = 16,
    imgsz: int = 640,
    device: str = "0",
    project: str = "runs/train",
    name: str = "counterfeit_detector",
    save_dir: str = "models"
) -> Path:
    """Train YOLOv8 model for counterfeit shoe detection."""

    # Validate data.yaml exists
    data_path = Path(data_yaml)
    if not data_path.exists():
        raise FileNotFoundError(f"Data config not found: {data_yaml}")

    logger.info("Loading data configuration...")
    with open(data_path, 'r') as f:
        data_config = yaml.safe_load(f)

    logger.info(f"Dataset classes: {data_config.get('names', [])}")
    logger.info(f"Number of classes: {data_config.get('nc', 'unknown')}")

    # Initialize YOLO model
    logger.info(f"Initializing {model} model...")
    yolo_model = YOLO(model)

    # Train the model
    logger.info("Starting training...")
    logger.info(f"Epochs: {epochs}, Batch: {batch}, Image size: {imgsz}, Device: {device}")

    results = yolo_model.train(
        data=str(data_path),
        epochs=epochs,
        batch=batch,
        imgsz=imgsz,
        device=device,
        project=project,
        name=name,
        save=True,
        plots=True,
        verbose=True
    )

    # Save best model to models directory
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    best_model_path = Path(project) / name / "weights" / "best.pt"
    if best_model_path.exists():
        final_path = save_path / "best.pt"
        import shutil
        shutil.copy(best_model_path, final_path)
        logger.info(f"Best model saved to: {final_path}")
        return final_path
    else:
        logger.warning("Best model weights not found in expected location")
        return best_model_path


# Start training
model_path = train_yolo(
    data_yaml='counterfeit-nike-shoes-detection-2/data.yaml',
    model='yolov8n.pt',
    epochs=40,
    batch=16,
    imgsz=640,
    device='0',
    project='runs/train',
    name='counterfeit_detector',
    save_dir='models'
)

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=counterfeit-nike-shoes-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=counterfeit_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspec